In [1]:
#install pyspark library
!pip install pyspark

In [2]:
#import pyspark
import pyspark


In [3]:
#import sparkSession
from pyspark.sql import SparkSession

In [4]:
#Creating a sparkSession object and providing appName 
spark=SparkSession.builder.appName("partitions").getOrCreate()

In [5]:
#to create dataframe form External datasets
%time AirlineDF = spark.read.option("header", "true").csv(r"C:\Users\User\Downloads\partitions\Data\*")

Wall time: 20.2 s


In [6]:
%time AirlineDF.show(1)

+----+-------+-----+----------+---------+----------+-------------+---------+-------+-------+---------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+---------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+--------------

In [ ]:
spark.stop()

In [9]:
#To check what data in created dataframe 
%time AirlineDF.take(1)

Wall time: 228 ms


[Row(Year='2005', Quarter='1', Month='1', DayofMonth='1', DayOfWeek='6', FlightDate='2005-01-01', UniqueCarrier='AA', AirlineID='19805', Carrier='AA', TailNum='N338AA', FlightNum='1', OriginAirportID='12478', OriginAirportSeqID='1247801', OriginCityMarketID='31703', Origin='JFK', OriginCityName='New York, NY', OriginState='NY', OriginStateFips='36', OriginStateName='New York', OriginWac='22', DestAirportID='12892', DestAirportSeqID='1289201', DestCityMarketID='32575', Dest='LAX', DestCityName='Los Angeles, CA', DestState='CA', DestStateFips='06', DestStateName='California', DestWac='91', CRSDepTime='0900', DepTime='0901', DepDelay='1.00', DepDelayMinutes='1.00', DepDel15='0.00', DepartureDelayGroups='0', DepTimeBlk='0900-0959', TaxiOut='27.00', WheelsOff='0928', WheelsOn='1206', TaxiIn='8.00', CRSArrTime='1210', ArrTime='1214', ArrDelay='4.00', ArrDelayMinutes='4.00', ArrDel15='0.00', ArrivalDelayGroups='0', ArrTimeBlk='1200-1259', Cancelled='0.00', CancellationCode=None, Diverted='0.0

In [10]:
#Calculate total rows in dataframe 
AirlineDF.count()

5939653

In [11]:
#calculate total columns in dataframe
len(AirlineDF.columns)

110

<h1> Partition on dataframe </h1>
<h1> reduce and increase no. of partition using coalesce() and repartion() </h1>
<h1> here we are partition  data according to size or no. of rows </h1>

In [12]:
#to check how many partition in current AirlineDF dataframe
AirlineDF.rdd.getNumPartitions()

22

In [14]:
#write dataframe in csv file and cross check no of partition
%time AirlineDF.write.mode("overwrite").csv("flight/airdata", header=True)

Wall time: 2min 25s


In [15]:
#reduce the no of partion from 22 to 5 here we are using coalesce() function 
#using coalesce() you can only reduce the no. of partition not increase no. of partition
%time AirlineDF1 = AirlineDF.coalesce(5)

Wall time: 79 ms


In [16]:
#to check how many partition in current AirlineDF1 dataFrame
AirlineDF1.rdd.getNumPartitions()

5

In [17]:
#write dataframe in csv file and cross check no of partition
%time AirlineDF1.write.mode("overwrite").csv("flight/airline_5", header=True)

Wall time: 1min 38s


In [18]:
#To increase the no of partition from 5 to 10 here we are using repartition() function
%time AirlineDF2 = AirlineDF1.repartition(10)

Wall time: 11 ms


In [19]:
#to check how many partition in current AirlineDF2 dataframe
AirlineDF2.rdd.getNumPartitions()

10

In [20]:
#write dataframe in csv file and cross check no of partition 
%time AirlineDF2.write.mode("overwrite").csv("flight/airline_10", header=True)

Wall time: 7min 25s


<h1> partition dataframe by perticular column </h1>

<h1> here we are using partitionBy() </h1>



In [21]:
#partition data by one column 
%time AirlineDF.write.option("header", True) \
        .partitionBy('Year') \
        .mode("overwrite") \
        .csv("flight/airline_year")

Wall time: 10min 57s
Parser   : 134 ms


In [22]:
#partition data by multiple columns
%time AirlineDF.write.option("header", True) \
        .partitionBy("Year","UniqueCarrier") \
        .mode("overwrite") \
        .csv("flight/airline_multiple")

Wall time: 11min 40s


<h1> Read partition data in pyspark with specific columns </h1>

In [23]:
#year 2015 and UniqueCarrier All
%time AirlineDF4 = spark.read.option("basePath","flight/airline_multiple") \
        .csv("flight/airline_multiple/Year=2015/UniqueCarrier=*")

Wall time: 2.7 s


In [24]:
#To check
%time AirlineDF4.select("Year","UniqueCarrier").show()

+----+-------------+
|Year|UniqueCarrier|
+----+-------------+
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
|2015|           WN|
+----+-------------+
only showing top 20 rows

Wall time: 215 ms


In [25]:
%time AirlineDF4.select("Year","UniqueCarrier").take(5)

Wall time: 525 ms


[Row(Year=2015, UniqueCarrier='WN'),
 Row(Year=2015, UniqueCarrier='WN'),
 Row(Year=2015, UniqueCarrier='WN'),
 Row(Year=2015, UniqueCarrier='WN'),
 Row(Year=2015, UniqueCarrier='WN')]

In [7]:
#all Year and UniqueCarrier
%time AirlineDF5 = spark.read.option("basePath", "flight/airline_multiple").csv("flight/airline_multiple/Year=*/UniqueCarrier=*")

Wall time: 3min 45s


In [28]:
%time AirlineDF5.select("Year","UniqueCarrier").show()

+----+-------------+
|Year|UniqueCarrier|
+----+-------------+
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
|2008|           WN|
+----+-------------+
only showing top 20 rows

Wall time: 1.09 s


In [29]:
#perform on partition data
%time AirlineDF5.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+-------------+------+
|UniqueCarrier| count|
+-------------+------+
|           AA|538352|
|           EV|391570|
|           UA|406417|
|           NW|163593|
|           B6|179952|
|           HP| 16256|
|           NK|  8745|
|   2014-07-14|    10|
|   2019-06-15|    12|
|   2011-12-13|     8|
|   2015-03-28|    14|
|   2014-01-23|    14|
|   2019-10-09|     6|
|   2015-07-06|    20|
|   2018-08-02|    10|
|   2018-10-25|     8|
|   2015-08-07|     8|
|   2019-01-27|    12|
|   2008-11-14|    10|
|   2009-08-09|     6|
+-------------+------+
only showing top 20 rows

Wall time: 1min 34s


In [31]:
#without partition
%time AirlineDF.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

+-------------+-------+
|UniqueCarrier|  count|
+-------------+-------+
|           UA| 406402|
|           AA| 538341|
|           NW| 163584|
|           B6| 179940|
|           DL| 568787|
|           US| 403807|
|           DH|  16279|
|           AS| 133808|
|           CO| 159640|
|           WN|1025308|
|           EV| 391554|
|           YV| 147059|
|           OO| 523534|
|           OH| 121143|
|           KH|   7903|
|           9E| 111730|
|           F9|  67129|
|           MQ| 424852|
|           HA|  59225|
|           XE| 212260|
+-------------+-------+
only showing top 20 rows

Wall time: 15.3 s


<h1> using repartition and partitionBy together </h1>

In [32]:
#Use repartition() and partitionBy() together
%time AirlineDF.repartition(3) \
        .write.option("header",True) \
        .partitionBy("Year") \
        .mode("overwrite") \
        .csv("flight/airline5")

#to check what happen when both are applying together

Wall time: 11min 27s


<h1> partitionBy() control number of partitions </h1>

In [33]:
%time AirlineDF.write.option("header",True) \
        .option("maxRecordPerFile", 1000000) \
        .partitionBy("Year") \
        .mode("overwrite") \
        .csv("flight/airline_max_records")
#to check what happen when applying maxRecordsPerFile is one million


Wall time: 11min 5s


<h1> with partition </h1>

In [10]:
#calculate total no of flights and group data by Carrier 
%time AirlineDF5.select('Year','UniqueCarrier') \
.filter((AirlineDF5.Year >= 2015)).groupBy('UniqueCarrier') \
.count().withColumnRenamed('count', 'Total Flights').orderBy('UniqueCarrier').show()

+-------------+-------------+
|UniqueCarrier|Total Flights|
+-------------+-------------+
|   1987-10-01|            8|
|   1987-10-02|           14|
|   1987-10-03|            6|
|   1987-10-04|            4|
|   1987-10-05|           14|
|   1987-10-06|            4|
|   1987-10-07|            6|
|   1987-10-08|            8|
|   1987-10-09|            8|
|   1987-10-10|           12|
|   1987-10-11|            4|
|   1987-10-12|           10|
|   1987-10-13|            2|
|   1987-10-14|            8|
|   1987-10-15|            4|
|   1987-10-16|            2|
|   1987-10-17|           10|
|   1987-10-18|            8|
|   1987-10-19|            6|
|   1987-10-20|            2|
+-------------+-------------+
only showing top 20 rows

Wall time: 1min 29s
Compiler : 161 ms


<h1> without partition </h1>

In [12]:
#calculate total no of flights and group data by Carrier 
%time AirlineDF.select('Year','UniqueCarrier') \
.filter((AirlineDF.Year >= 2015)).groupby('UniqueCarrier') \
.count().withColumnRenamed('count','Total Flights').orderBy('UniqueCarrier').show()

+-------------+-------------+
|UniqueCarrier|Total Flights|
+-------------+-------------+
|   1987-10-01|            4|
|   1987-10-02|            7|
|   1987-10-03|            3|
|   1987-10-04|            2|
|   1987-10-05|            7|
|   1987-10-06|            2|
|   1987-10-07|            3|
|   1987-10-08|            4|
|   1987-10-09|            4|
|   1987-10-10|            6|
|   1987-10-11|            2|
|   1987-10-12|            5|
|   1987-10-13|            1|
|   1987-10-14|            4|
|   1987-10-15|            2|
|   1987-10-16|            1|
|   1987-10-17|            5|
|   1987-10-18|            4|
|   1987-10-19|            3|
|   1987-10-20|            1|
+-------------+-------------+
only showing top 20 rows

Wall time: 20.6 s
